In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)
import nltk

In [2]:
t_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_first_train.csv',names=["text","label","NaN"], sep='\t')
t_data.pop('NaN')
t_data.head(9)

,text,label
0,Realme india product,not-Tamil
1,I also don't have tiktok hello and allnBut I'm using xiaomi,Hope_speech
2,Thalaivare..neengale inum one plus mobile vachurukinga...adha udachurunga,Non_hope_speech
3,Annee varanda thondai.. corona virus affect pannum.. neenga pathukonga. Pause koduth pasuga anne...,Hope_speech
4,5views but 18likes,Non_hope_speech
5,China phone vakathiga inu evanga ellam tiktok la video poduravagal nChina Phone vendam but Chin...,Non_hope_speech
6,Cisco webex meetings bro,Non_hope_speech
7,Yaaru madan gowriyaa ban panrenu sonnaaru? Yaarukita solringa,Non_hope_speech
8,Sabaash semma aarambam,Non_hope_speech


In [3]:
t_data['text']=t_data['text'].str.lower()       #changing into lower case       (remove and check acc too)

In [4]:
t_data['text']=t_data['text'].str.strip()       #remove white spaces
t_data['text']=t_data['text'].str.replace(r'\d+','')   #remove numbers
#t_data['text']=t_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
t_data['text']=t_data['text'].str.replace('[^\w\s]','')      #removing punct

In [5]:
#removing url if any
import re
def remove_URL(txt):
    url= re.compile(r"https?://\S+|www\.\S+")
    return url.sub(r"",txt)
t_data['text']=t_data['text'].apply(lambda x:remove_URL(x))

In [6]:
import numpy as np
from keras.utils.np_utils import to_categorical
from sklearn import preprocessing
labelEncode=preprocessing.LabelEncoder()
labelEncode.fit(t_data['label'])
print (labelEncode.classes_)
train_labelEncode=labelEncode.transform(t_data['label'])
label=to_categorical(np.asarray(train_labelEncode))
label

['Hope_speech' 'Non_hope_speech' 'not-Tamil']


array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [7]:
#counting labels      
 
t_data['label'].value_counts()
#must use something to balance data

Non_hope_speech    7872
Hope_speech        6327
not-Tamil          1961
Name: label, dtype: int64

In [8]:
#words in each line
 
t_data['totalwords'] = t_data['text'].str.count(' ') + 1
t_data

,text,label,totalwords
0,realme india product,not-Tamil,3
1,i also dont have tiktok hello and allnbut im using xiaomi,Hope_speech,11
2,thalaivareneengale inum one plus mobile vachurukingaadha udachurunga,Non_hope_speech,7
3,annee varanda thondai corona virus affect pannum neenga pathukonga pause koduth pasuga anne thon...,Hope_speech,16
4,views but likes,Non_hope_speech,3
...,...,...,...
16155,pm correcta erukum mg bro,Hope_speech,5
16156,intha karutha mudija varaikum neengalum ellarkittayum kondu poi sekkanum bro,Non_hope_speech,10
16157,and neenga adhiyavasiyam nu soldra apps like tiktok uc browser etc la oru mukkiyamana appe keday...,Non_hope_speech,20
16158,daii sekram mater ku vada,Hope_speech,5


In [9]:
import keras
from keras.preprocessing.text import Tokenizer
tok1 = Tokenizer(char_level=False, filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n', lower = True)

In [10]:
tok2=Tokenizer(char_level=True,filters ='!"$%&@()*+,-./:;”“<=>?[\\]^_`{|}~\t\n',lower=True)

In [11]:
tok1.fit_on_texts(t_data['text'])
words=len(tok1.word_counts)
words

32041

In [12]:
tok2.fit_on_texts(t_data['text'])

In [13]:
Charlist=tok2.word_index
#Charlist

In [14]:
wordlist=tok1.word_index
#wordlist

In [15]:
encode=tok1.texts_to_sequences(t_data['text'])
print(t_data['text'][5555])
encode[5555]

இநதய எலலப பகதயல சன சல அமபபத இநதய இரணவம அனமதகக கடத


[46, 7113, 2811, 21, 697, 17525, 46, 2761, 5015, 841]

In [16]:
##         Padding encoded sequence of words
from keras.preprocessing import sequence
max_length=20
padd = sequence.pad_sequences(encode, maxlen=max_length, padding='post')
padd

array([[  333,    10,   213, ...,     0,     0,     0],
       [   24,   364,   355, ...,     0,     0,     0],
       [ 9657,   919,   172, ...,     0,     0,     0],
       ...,
       [   49,    30, 32038, ...,   523,   227,  1157],
       [ 2947, 32041,  4225, ...,     0,     0,     0],
       [    1,  1116,   258, ...,     0,     0,     0]], dtype=int32)

In [17]:
# importing libraries for creating model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding,MaxPool1D
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D,LSTM,Bidirectional

#Embedding layer

In [18]:
Embedding_Layer=Embedding(input_dim = words + 1, output_dim = 300,input_length=max_length)

#CNN

In [19]:
model = Sequential()
model.add(Embedding_Layer)
model.add(Conv1D(filters =256,kernel_size=2, activation='relu'))
model.add(MaxPooling1D(2))
model.add(Conv1D(filters =512,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(3,padding='same'))
model.add(Flatten())
model.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model.add(Dense(3, activation = 'sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 300)           9612600   
_________________________________________________________________
conv1d (Conv1D)              (None, 19, 256)           153856    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 9, 256)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 7, 512)            393728    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 3, 512)            0         
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                4

#LSTM

In [20]:
model_1 = Sequential()
model_1.add(Embedding_Layer)
model_1.add(LSTM(256,return_sequences=False))

model_1.add(Flatten())
model_1.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model_1.add(Dense(3, activation = 'sigmoid'))
model_1.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 300)           9612600   
_________________________________________________________________
lstm (LSTM)                  (None, 256)               570368    
_________________________________________________________________
flatten_1 (Flatten)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 10,191,291
Trainable params: 10,191,291
Non-trainable params: 0
_________________________________________________________________


#Bidirectional

In [21]:
model_2 = Sequential()
model_2.add(Embedding_Layer)
model_2.add(Bidirectional(LSTM(256,return_sequences=False)))

model_2.add(Flatten())
model_2.add(Dense(32, activation = 'relu'))
#model.add(Dense(4, activation = 'relu'))
model_2.add(Dense(3, activation = 'sigmoid'))
model_2.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 300)           9612600   
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               1140736   
_________________________________________________________________
flatten_2 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 32)                16416     
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 99        
Total params: 10,769,851
Trainable params: 10,769,851
Non-trainable params: 0
_________________________________________________________________


#compile

In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_1.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

#Training models

In [23]:
model.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
253/253 [==============================] - 60s 232ms/step - loss: 0.5382 - acc: 0.5220
Epoch 2/20
253/253 [==============================] - 58s 230ms/step - loss: 0.3716 - acc: 0.7444
Epoch 3/20
253/253 [==============================] - 59s 231ms/step - loss: 0.1804 - acc: 0.8907
Epoch 4/20
253/253 [==============================] - 59s 231ms/step - loss: 0.1121 - acc: 0.9334
Epoch 5/20
253/253 [==============================] - 59s 233ms/step - loss: 0.0807 - acc: 0.9503
Epoch 6/20
253/253 [==============================] - 59s 232ms/step - loss: 0.0683 - acc: 0.9550
Epoch 7/20
253/253 [==============================] - 58s 230ms/step - loss: 0.0563 - acc: 0.9636
Epoch 8/20
253/253 [==============================] - 59s 231ms/step - loss: 0.0506 - acc: 0.9667
Epoch 9/20
253/253 [==============================] - 59s 235ms/step - loss: 0.0452 - acc: 0.9692
Epoch 10/20
253/253 [==============================] - 60s 235ms/step - loss: 0.0403 - acc: 0.9735
Epoch 11/20
253/253

In [24]:
model_1.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
253/253 [==============================] - 75s 288ms/step - loss: 0.4289 - acc: 0.6827
Epoch 2/20
253/253 [==============================] - 72s 285ms/step - loss: 0.1713 - acc: 0.8967
Epoch 3/20
253/253 [==============================] - 72s 284ms/step - loss: 0.1290 - acc: 0.9177
Epoch 4/20
253/253 [==============================] - 72s 283ms/step - loss: 0.1077 - acc: 0.9307
Epoch 5/20
253/253 [==============================] - 71s 282ms/step - loss: 0.0992 - acc: 0.9344
Epoch 6/20
253/253 [==============================] - 71s 281ms/step - loss: 0.0918 - acc: 0.9375
Epoch 7/20
253/253 [==============================] - 71s 281ms/step - loss: 0.0839 - acc: 0.9421
Epoch 8/20
253/253 [==============================] - 71s 282ms/step - loss: 0.0813 - acc: 0.9411
Epoch 9/20
253/253 [==============================] - 72s 284ms/step - loss: 0.0753 - acc: 0.9461
Epoch 10/20
253/253 [==============================] - 71s 282ms/step - loss: 0.0737 - acc: 0.9491
Epoch 11/20
253/253

In [ ]:
model_2.fit(padd,label,epochs=20,verbose=1,batch_size=64)

Epoch 1/20
253/253 [==============================] - 112s 428ms/step - loss: 0.2755 - acc: 0.8118
Epoch 2/20
253/253 [==============================] - 107s 423ms/step - loss: 0.0934 - acc: 0.9399
Epoch 3/20
253/253 [==============================] - 108s 425ms/step - loss: 0.0747 - acc: 0.9465
Epoch 4/20
253/253 [==============================] - 106s 418ms/step - loss: 0.0638 - acc: 0.9557
Epoch 5/20
253/253 [==============================] - 108s 428ms/step - loss: 0.0623 - acc: 0.9547
Epoch 6/20
253/253 [==============================] - 107s 424ms/step - loss: 0.0620 - acc: 0.9537
Epoch 7/20
253/253 [==============================] - 108s 427ms/step - loss: 0.0547 - acc: 0.9631
Epoch 8/20
253/253 [==============================] - 107s 423ms/step - loss: 0.0490 - acc: 0.9628
Epoch 9/20
253/253 [==============================] - 106s 420ms/step - loss: 0.0539 - acc: 0.9619
Epoch 10/20
253/253 [==============================] - 105s 417ms/step - loss: 0.0482 - acc: 0.9680
Epoch 11/

#Development data

In [ ]:
d_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_first_dev.csv',names=['text','label','nan'],sep='\t')

In [ ]:
d_data=d_data[["text","label"]]
d_data

,text,label
0,Mg bro eve 6 o clock video post pannuga,Hope_speech
1,8.00 pm bro plss,not-Tamil
2,Bro ivan kitta sonna . Ivan change' pannidava poran,Hope_speech
3,@Varnajalam Mini Crafts adhum crrct thaan Akka 🤣,Non_hope_speech
4,எல்லா குழந்தைகளும் நல்ல எண்ணம்,Hope_speech
...,...,...
2013,Share karo app use panlama,Non_hope_speech
2014,@R SWAGxOP kelattu kapothi maari nee pesathada mundam kelavaa,Non_hope_speech
2015,Yow yaean yaa kelappi vidringa ipd,Non_hope_speech
2016,Hey bro. Ur doing amazing...,not-Tamil


In [ ]:
#processing as training set

d_data['text']=d_data['text'].str.lower()       #changing into lower case       (remove and check acc too)
d_data['text']=d_data['text'].str.strip()       #remove white spaces
d_data['text']=d_data['text'].str.replace(r'\d+','')   #remove numbers
#d_data['text']=d_data['text'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))  #removing emoji
d_data['text']=d_data['text'].str.replace('[^\w\s]','')      #removing punct
d_data['text']=d_data['text'].apply(lambda x:remove_URL(x))
#d_data['text'] = d_data['text'].apply(lambda x: remove_sw(x))


In [ ]:
encoded1 =tok1.texts_to_sequences(d_data['text'])
print(d_data['text'][0])
encoded1[0]

mg bro eve  o clock video post pannuga


[5, 1, 2446, 190, 262, 6, 75, 187]

In [ ]:
padded = sequence.pad_sequences(encoded1, maxlen=max_length, padding='post')
padded[0]

array([   5,    1, 2446,  190,  262,    6,   75,  187,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [ ]:
val_labelEncode=labelEncode.transform(d_data['label'])
val_label=to_categorical(np.asarray(val_labelEncode))
val_label[0]

array([1., 0., 0.], dtype=float32)

#prediction

In [ ]:
# devlopment prediction of CNN

dev_predictions = model.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
dev_predictions1 = np.zeros_like(dev_predictions)
dev_predictions1[np.arange(len(dev_predictions)), dev_predictions.argmax(1)] = 1
#print(dev_predictions)
#print(dev_predictions1)

In [ ]:
# devlopment prediction of Lstm

l_dev_predictions = model_1.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
l_dev_predictions1 = np.zeros_like(l_dev_predictions)
l_dev_predictions1[np.arange(len(l_dev_predictions)), l_dev_predictions.argmax(1)] = 1
#print(l_dev_predictions)
#print(l_dev_predictions1)

In [ ]:
# devlopment prediction of BiLSTM

b_dev_predictions = model_2.predict(padded)
# Finding out of two output neurons which one has highest probability
# It will return us the predicted class
# It will convert probability into final class 1 and 0
b_dev_predictions1 = np.zeros_like(b_dev_predictions)
b_dev_predictions1[np.arange(len(b_dev_predictions)), b_dev_predictions.argmax(1)] = 1
#print(b_dev_predictions)
#print(b_dev_predictions1)

#Acuracy report

In [ ]:
#Accuracy CNN
from sklearn.metrics import classification_report
print(classification_report(val_label,dev_predictions1))

              precision    recall  f1-score   support

           0       0.47      0.48      0.48       757
           1       0.61      0.60      0.60       998
           2       0.53      0.51      0.52       263

   micro avg       0.54      0.54      0.54      2018
   macro avg       0.53      0.53      0.53      2018
weighted avg       0.54      0.54      0.54      2018
 samples avg       0.54      0.54      0.54      2018



In [ ]:
#Accuracy Lstm
print(classification_report(val_label,l_dev_predictions1))

              precision    recall  f1-score   support

           0       0.48      0.57      0.52       757
           1       0.62      0.54      0.58       998
           2       0.50      0.49      0.50       263

   micro avg       0.54      0.54      0.54      2018
   macro avg       0.53      0.53      0.53      2018
weighted avg       0.55      0.54      0.54      2018
 samples avg       0.54      0.54      0.54      2018



In [ ]:
#Accuracy BiiLstm
print(classification_report(val_label,b_dev_predictions1))

              precision    recall  f1-score   support

           0       0.48      0.50      0.49       757
           1       0.62      0.54      0.57       998
           2       0.43      0.61      0.51       263

   micro avg       0.53      0.53      0.53      2018
   macro avg       0.51      0.55      0.52      2018
weighted avg       0.54      0.53      0.53      2018
 samples avg       0.53      0.53      0.53      2018



#Testing Data

In [ ]:
t_data=pd.read_csv('/content/drive/MyDrive/Ankit/dec 2020 codalab/Hope speech/Tamil/tamil_hope_test.csv',names=['text'],sep=',')
t_data

FileNotFoundError: ignored